In [ ]:
!pip install tensorboard datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

import numpy as np
import pandas as pd

from datasets import load_dataset
from PIL import Image

In [ ]:
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data_dir = "./data"
celeba = load_dataset("tpremoli/CelebA-attrs", split="all", cache_dir=data_dir)

df = celeba.to_pandas()

attr_names = df.columns[1:-1]
image_col_name = df.columns[0]

attributes = df[attr_names].values.astype(np.float32)

inputs = df[image_col_name].values

targets = (attributes + 1) / 2

train_percentage = 0.8
val_percentage = 0.1
test_percentage = 1 - train_percentage - val_percentage

training_amount = int(train_percentage * len(inputs))
validation_amount = int(val_percentage * len(inputs))
testing_amount = int(train_percentage * len(inputs))

train_inputs = inputs[0:training_amount]
val_inputs = inputs[training_amount:training_amount+validation_amount]
test_inputs = inputs[training_amount+validation_amount:]


train_targets = targets[0:training_amount]
val_targets = targets[training_amount:training_amount+validation_amount]
test_targets = targets[training_amount+validation_amount:]

train_targets = torch.tensor(train_targets, dtype=torch.float32).to(device)
val_targets = torch.tensor(val_targets, dtype=torch.float32).to(device)
test_targets = torch.tensor(test_targets, dtype=torch.float32).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/162770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/19962 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/19867 [00:00<?, ? examples/s]

In [ ]:
from PIL import Image
from io import BytesIO

# Define custom transform
custom_transform = transforms.Compose([
    transforms.CenterCrop((178, 178)),
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

from torch.utils.data import DataLoader, Dataset

class CelebADataset(Dataset):
    def __init__(self, images, targets, transform=None):
        self.images = images  # List of image paths or byte data dictionaries
        self.targets = targets  # Corresponding targets
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_data = self.images[idx]

        if isinstance(image_data, dict):  # If image data is stored as a dict (e.g., {'bytes': ...})
            img_bytes = image_data['bytes']
            image = Image.open(BytesIO(img_bytes))
        else:  # Otherwise, assume it's a file path
            image = Image.open(image_data)

        if self.transform:
            image = self.transform(image)

        target = self.targets[idx]
        return image, target

In [ ]:
import torchvision.models as models

# Load a pre-trained ResNet model
resnet_model = models.resnet50(pretrained=True)

# Modify the final layer to match the number of attributes (40 in this case)
num_features = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_features, 40)

# Move model to the appropriate device
resnet_model = resnet_model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 131MB/s]


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100):
    writer = SummaryWriter(log_dir="./tensorboard_logs")

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for batch_idx, (images, targets) in enumerate(train_loader):
            images, targets = images.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        eval_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        with torch.no_grad():
            for batch_idx, (images, targets) in enumerate(val_loader):
                images, targets = images.to(device), targets.to(device)

                outputs = model(images)
                loss = criterion(outputs, targets)
                eval_loss += loss.item()

                predictions = torch.sigmoid(outputs)
                predicted_labels = (predictions > 0.5).float()
                correct_predictions += (predicted_labels == targets).sum().item()
                total_predictions += targets.numel()

        avg_eval_loss = eval_loss / len(val_loader)
        accuracy = correct_predictions / total_predictions

        # Log to TensorBoard
        writer.add_scalar("Loss/Train", avg_train_loss, epoch)
        writer.add_scalar("Loss/Eval", avg_eval_loss, epoch)
        writer.add_scalar("Accuracy", accuracy, epoch)
        writer.add_scalar("Learning Rate", scheduler.get_last_lr()[0], epoch)

        print(f"Epoch {epoch + 1}, Train Loss: {avg_train_loss:.4f}, Eval Loss: {avg_eval_loss:.4f}, Accuracy: {accuracy:.4f}")

        # Step the scheduler
        scheduler.step()

    # Close the TensorBoard writer
    writer.close()

    return model

In [ ]:
# Custom dataset and dataloaders for ResNet and CNN
train_dataset = CelebADataset(train_inputs, train_targets, transform=custom_transform)
val_dataset = CelebADataset(val_inputs, val_targets, transform=custom_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

optimizer = optim.AdamW(resnet_model.parameters(), lr=1e-6, weight_decay=0.01)
scheduler = ExponentialLR(optimizer, gamma=0.99)
trained_resnet_model = train_model(resnet_model, train_loader, val_loader, nn.BCEWithLogitsLoss(), optimizer, scheduler)

torch.save(trained_resnet_model.state_dict(), "./trained/AnoMed/resnet_model.pth")


Epoch 1, Train Loss: 0.0931, Eval Loss: 0.2404, Accuracy: 0.9078
Epoch 2, Train Loss: 0.0922, Eval Loss: 0.2415, Accuracy: 0.9080
Epoch 3, Train Loss: 0.0918, Eval Loss: 0.2428, Accuracy: 0.9079
Epoch 4, Train Loss: 0.0913, Eval Loss: 0.2453, Accuracy: 0.9078
Epoch 5, Train Loss: 0.0910, Eval Loss: 0.2436, Accuracy: 0.9076
Epoch 6, Train Loss: 0.0908, Eval Loss: 0.2445, Accuracy: 0.9077
Epoch 7, Train Loss: 0.0905, Eval Loss: 0.2449, Accuracy: 0.9077
Epoch 8, Train Loss: 0.0902, Eval Loss: 0.2447, Accuracy: 0.9079
Epoch 9, Train Loss: 0.0900, Eval Loss: 0.2462, Accuracy: 0.9077
Epoch 10, Train Loss: 0.0897, Eval Loss: 0.2458, Accuracy: 0.9075


KeyboardInterrupt: 